In [2]:
import torch, sys, glob, os, math
from pathlib import Path
import numpy as np
import torch.multiprocessing as mp
mp.set_start_method('fork')
from multiprocessing import Pool
from copy import deepcopy


/home/is/armin-sa/miniconda3/envs/pytorch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Binarize Training Data

In [3]:
n = 16
cwd = os.getcwd()
prep_train_data_name = "ru_small_id.txt"


In [4]:
%cd data
!split -n l/{n} {prep_train_data_name} chunk_
%cd {cwd}

/project/cl-work2/armin-sa/resources/cc100-rus
/home/is/armin-sa/Projects/lm


In [5]:
def binarize_chunk(f_in_path):
    with open(f_in_path, "r") as f_in, open(
        "data/prep_" + Path(f_in_path).stem, "wb"
    ) as f_out:
        for line in f_in:
            arr = np.fromstring(line, dtype=np.int32, sep=" ")
            f_out.write(
                len(arr).to_bytes(2, byteorder=sys.byteorder, signed=False)
                + arr.tobytes()
            )


In [6]:
with Pool(processes=n) as p:
    p.map(binarize_chunk, glob.glob("data/chunk_*"))


In [7]:
%cd data
!rm chunk_*
!cat prep_chunk_* > ru_small.bin
!rm prep_chunk_*
%cd {cwd}

/project/cl-work2/armin-sa/resources/cc100-rus
/home/is/armin-sa/Projects/lm


In [8]:
sample_locs = [0]
with open("data/ru_small_id.txt", "r") as f_in:
    line = f_in.readline()
    while line:
        f_in.tell() #returns the location of the next line
        line = f_in.readline()
        sample_locs.append(f_in.tell())

sample_locs = np.array(sample_locs[:-1])


In [9]:
with open("data/ru_small_id.txt", "r") as f:
    f.seek(sample_locs[2])
    l = f.readline()
l


'1 442 85 4 66 1056 13 55 307 24 19847 52 28220 197 40 2984 6863 10600 7469 369 4 46 413 59 2984 7 496 5367 560 24 1591 1076 560 161 6 543 1952 11271 520 7469 369 5 2\n'

In [10]:
# def _gen():
#     with open("data/ru_small.bin", "rb") as f_in:
#         while count := int.from_bytes(f_in.read(2), byteorder=sys.byteorder, signed=False):
#             arr = torch.frombuffer(f_in.read(count * 4), count=count, dtype=torch.int)
#             yield arr


In [11]:
# a =_gen()
# next(a)


In [16]:
import linecache
class CC100DataSet(torch.utils.data.Dataset):
    def __init__(self, file_path, num_workers):
        super().__init__()
        self.path=file_path
        self.sample_locs = deepcopy(sample_locs)
        self.file_handles = [open(file_path, "r") for _ in range(num_workers)]
        self.file_handles.append(open(file_path, "r"))

    def __len__(self):
        return len(self.sample_locs)

    def __getitem__(self, idx):
        worker_info = torch.utils.data.get_worker_info()
        if worker_info is None:
            worker_id = 0
        else:
            worker_id = worker_info.id
        # self.file_handles[worker_id].seek(self.sample_locs[idx])
        
        line = linecache.getline(self.path, idx)
        arr = np.fromstring(line, dtype=np.int32, sep=" ")
        # if len(line) > 0:
            # arr = torch.tensor(list(map(int, line.split(' '))), dtype=torch.int16)
            # x = torch.arange(200)
            # return arr
        return np.copy(arr)
        # return torch.arange(200)

def collate_wrapper(batch):
    return batch
    # return list(map(lambda x: torch.tensor(list(map(int, x.split(' '))), dtype=torch.int16), batch))
    # return torch.utils.data.default_convert(batch)


num_workers = 4
ds = CC100DataSet("data/ru_small_id.txt", num_workers)
data_loader = torch.utils.data.DataLoader(
    ds,
    num_workers=num_workers,
    shuffle=False,
    batch_size=64,
    collate_fn=collate_wrapper,
    # prefetch_factor=1,
    # persistent_workers=True,
    # pin_memory=True,
)


In [17]:
# x=iter(data_loader)
# print(next(x))
# x=iter(data_loader)
# print(next(x))
import time
c = 0
# l = [0]
for _ in data_loader:
    # l[0]=_
    c+=1
    # time.sleep(0.1)

In [18]:
# with open("data/ru_small.bin", "rb") as f:
#     f.seek(sample_locs[0])
#     count = int.from_bytes(f.read(2), byteorder=sys.byteorder, signed=False)
#     arr = np.fromstring(f.read(count * 4), count=count, dtype=np.int32)
# arr

In [19]:
print(c)


15182


In [20]:
def run():
    pass

In [22]:
%prun  run()

         4 function calls in 0.000 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 723293958.py:1(run)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}